In [ ]:
dynamic monte carlo

finance: focus on optimal stopping, am or ac
ls+tsi: mc for optimal stopping
several: pde for am
seq dec: stoch control>pde low dim
    
rl: focus on seq dec
dyn prg = pde too slow
sample appr
value method = mc = ls
pg also exists in finance, see eg jaeckel

finance requires high reliability+prec+speed (for intraday risk calcs) in a constantly changing model (because mkt data change)

new appr: dyn mc
compare to heuristic (geodesic) and dqn (stable baseline)


# Dynamic Monte Carlo
Optimal stopping problems have been studied in option pricing in finance. We analyzed here a product with sequential decision making by a mixture approach of dynamic programming and reinforcement learning. We show that it gives a better policy than heuristic rules and pure reinforcement learning approaches.

## Introduction

## Review



## Model



## Dynamic Monte Carlo

## Comparison to other approaches

## Conclusion


In [13]:
import numpy as np
from numpy.linalg import eig
import pandas as pd
import py_lets_be_quickly_rational
import time
from scipy import interpolate
from scipy.stats import norm
from scipy.optimize import brentq
from matplotlib import pyplot as plt

0) intro
1) review > only optimal stopping
2) model > stoch correl; no resim because chance player always the same; but exploration question remains; check with deepmind paper
2) geodesic+grad descent
3) dqn
4) mcts

In [14]:
def BS(F,K,T,vol):
    d1=(np.log(F/K)+0.5*vol**2*T)/(vol*T**0.5)
    d2=d1-vol*T**0.5
    return F*norm.cdf(d1) - K*norm.cdf(d2)

def IV(price,F,K,T):
    return brentq(lambda v: BS(F,K,T,v)-price, 0.01, 0.5)

In [15]:
# s0, v0, kappa, theta, alpha=vov, rho
modelParam = np.array([
    [50.0, 0.25**2, 3.0, 0.3**2, 0.4, -0.6],
    [50.0, 0.35**2, 3.0, 0.25**2, 0.4, -0.5],
    [50.0, 0.3**2, 2.0, 0.3**2, 0.3, -0.7],
])

n = len(modelParam)

# constraint 2 theta vbar > alpha**2
for i in range(n):
    _, _, theta, vbar, alpha, _ = modelParam[i]
    print(2*theta*vbar-alpha**2)

0.38
0.21499999999999997
0.27


In [16]:
C = np.array([
    [1.0, 0.7, 0.3],
    [0.7, 1.0, 0.5],
    [0.3, 0.5, 1.0]
])

L = np.linalg.cholesky(C)
print(L.T)
print(np.matmul(L, L.T))
print(C-np.matmul(L, L.T))
    
for i in range(3):
    b=np.random.multivariate_normal(np.zeros(3),0.02**2*np.eye(3))
    a = L[i,:] + b
    L[i,:] = a / np.linalg.norm(a)
print(np.matmul(L, L.T))

[[1.         0.7        0.3       ]
 [0.         0.71414284 0.40608122]
 [0.         0.         0.86319062]]
[[1.  0.7 0.3]
 [0.7 1.  0.5]
 [0.3 0.5 1. ]]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.11022302e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -2.22044605e-16]]
[[1.         0.71032838 0.32089553]
 [0.71032838 1.         0.4918853 ]
 [0.32089553 0.4918853  1.        ]]


In [17]:
def xi(a,b,c,d,k):
    r1 = np.cos(k*np.pi*(d-a)/(b-a))*np.exp(d) - np.cos(k*np.pi*(c-a)/(b-a))*np.exp(c)
    r2 = k*np.pi/(b-a)*np.sin(k*np.pi*(d-a)/(b-a))*np.exp(d) - k*np.pi/(b-a)*np.sin(k*np.pi*(c-a)/(b-a))*np.exp(c)
    return (r1+r2) / (1+(k*np.pi/(b-a))**2)

def vpsi(a,b,c,d,ks):
    ks = np.maximum(ks,1)
    r = (np.sin(ks*np.pi*(d-a)/(b-a)) - np.sin(ks*np.pi*(c-a)/(b-a)))*(b-a)/(ks*np.pi)
    r[0] = d-c
    return r

def vv_put_orig(a, b, ks):
    r = 2 / (b-a) * (-xi(a,b,a,0,ks)+vpsi(a,b,a,0,ks))
    r[0] = r[0] / 2.0
    return r

def phi_heston2_pre(omega, kappa,theta, alpha, rho,T):
    z=omega
    tau=T
    v=V0
    vbar=theta
    lamb=kappa
    eta=alpha
    a0=lamb-rho*eta*z*1j
    gamma=np.power(eta**2*(z**2+z*1j)+a0*a0, 0.5)
    G=(a0-gamma)/(a0+gamma)
    a1=1.0/eta/eta*((1-np.exp(-gamma*tau))/(1-G*np.exp(-gamma*tau)))*(a0-gamma)
    a2=lamb*vbar/eta/eta*(tau*(a0-gamma)-2*np.log((1-G*np.exp(-gamma*tau))/(1-G))) #c*vbar
    return a1, a2

def phi_heston2(omega, kappa,theta, alpha, rho,V0,T):
    a1, a2 = phi_heston2_pre(omega, kappa,theta, alpha, rho,T)
    return np.exp(V0*a1+a2)

def phi_heston2_post(a,V0):
    a1, a2 = a
    return np.exp(V0*a1+a2)

def cumulants(kappa, theta, V0, t):
    c1 = (1-np.exp(-kappa*t))*(theta-V0)/(2*kappa)-theta*t/2
    c21 = V0 / (4*kappa**3) * (4*kappa**2*(1+(rho*alpha*t-1)*np.exp(-kappa*t))
                             +kappa*(4*rho*alpha*(np.exp(-kappa*t)-1)-2*alpha**2*t*np.exp(-kappa*t))
                             +alpha**2*(1-np.exp(-2*kappa*t)))
    c22 = theta / (8*kappa**3) * (8*kappa**3*t - 8*kappa**2*(1+rho*alpha*t+(rho*alpha*t-1)*np.exp(-kappa*t))+\
                                  2*kappa*((1+2*np.exp(-kappa*t))*alpha**2*t+ 8*(1-np.exp(-kappa*t))*rho*alpha)+\
                                  alpha**2*(np.exp(-2*kappa*t)+4*np.exp(-kappa*t)-5))
    return c1, c21+c22

In [18]:
T=0.25
S0, V0, kappa, theta, alpha, rho = modelParam[2,:]
c1, c2 = cumulants(kappa, theta, V0, T)
c1, c2 = cumulants(kappa, theta, theta, T)
L=12
a=c1 - L*c2**0.5
b=c1 + L*c2**0.5
F=S0
N=50

In [19]:
Ks=np.linspace(0.9*F,1.1*F,11)
t1=time.perf_counter()
xs=np.log(F/Ks)
ks = np.array(range(N))
xxs = ks*np.pi/(b-a)

h = phi_heston2(xxs, kappa,theta,alpha,rho,V0,T) * vv_put_orig(a,b,ks)
res = np.zeros(len(Ks))
iv = np.zeros(len(Ks))
q=-1 #put
numIt=1
for i in range(len(xs)):
    res[i] = Ks[i] * np.sum(h * np.exp(1j * (xs[i] - a) * xxs)).real
    iv[i] = py_lets_be_quickly_rational.implied_volatility_from_a_transformed_rational_guess_with_limited_iterations(res[i], S0, Ks[i], T, q, numIt)
t2=time.perf_counter()
print("time:", t2-t1)

for i in range(len(Ks)):
    print(Ks[i], res[i], iv[i])
print("calls")
for i in range(len(Ks)):
    print(Ks[i], res[i] + F - Ks[i])

time: 0.0021704170000020895
45.0 1.0990571183547682 0.3118519979727549
46.0 1.3718263057523672 0.3086445823669359
47.0 1.6908850143843153 0.30547917513218253
48.0 2.059207635490383 0.30235601306908555
49.0 2.479053778995939 0.2992756051490698
50.0 2.9518551739744465 0.2962387265102957
51.0 3.478140049661096 0.2932464308338895
52.00000000000001 4.057501441078789 0.2903000729847739
53.00000000000001 4.688612457633232 0.28740132925934087
54.00000000000001 5.369287845599499 0.28455220444366564
55.00000000000001 6.09658752228287 0.28175502034055
calls
45.0 6.0990571183547715
46.0 5.371826305752364
47.0 4.690885014384314
48.0 4.059207635490381
49.0 3.47905377899594
50.0 2.951855173974444
51.0 2.478140049661093
52.00000000000001 2.0575014410787844
53.00000000000001 1.6886124576332264
54.00000000000001 1.3692878455994943
55.00000000000001 1.09658752228286


In [20]:
t1=time.perf_counter()
xs=np.log(F/Ks)
ks = np.array(range(N))
xxs = ks*np.pi/(b-a)

#h = phi_heston2(xxs, kappa,theta,alpha,rho,V0,T) * vv_put_orig(a,b,ks)
vv_po = vv_put_orig(a,b,ks)
heston_a = phi_heston2_pre(xxs, kappa,theta,alpha,rho,T)
h = phi_heston2_post(heston_a, V0) * vv_po
res = np.zeros(len(Ks))
iv = np.zeros(len(Ks))
q=-1 #put
numIt=1
for i in range(len(xs)):
    res[i] = Ks[i] * np.sum(h * np.exp(1j * (xs[i] - a) * xxs)).real
    iv[i] = py_lets_be_quickly_rational.implied_volatility_from_a_transformed_rational_guess_with_limited_iterations(res[i], S0, Ks[i], T, q, numIt)
t2=time.perf_counter()
print("time:", t2-t1)

for i in range(len(Ks)):
    print(Ks[i], res[i], iv[i])
print("calls")
for i in range(len(Ks)):
    print(Ks[i], res[i] + F - Ks[i])

time: 0.00264258400000017
45.0 1.0990571183547682 0.3118519979727549
46.0 1.3718263057523672 0.3086445823669359
47.0 1.6908850143843153 0.30547917513218253
48.0 2.059207635490383 0.30235601306908555
49.0 2.479053778995939 0.2992756051490698
50.0 2.9518551739744465 0.2962387265102957
51.0 3.478140049661096 0.2932464308338895
52.00000000000001 4.057501441078789 0.2903000729847739
53.00000000000001 4.688612457633232 0.28740132925934087
54.00000000000001 5.369287845599499 0.28455220444366564
55.00000000000001 6.09658752228287 0.28175502034055
calls
45.0 6.0990571183547715
46.0 5.371826305752364
47.0 4.690885014384314
48.0 4.059207635490381
49.0 3.47905377899594
50.0 2.951855173974444
51.0 2.478140049661093
52.00000000000001 2.0575014410787844
53.00000000000001 1.6886124576332264
54.00000000000001 1.3692878455994943
55.00000000000001 1.09658752228286


In [21]:
# https://kluge.in-chemnitz.de/tools/pricer/heston_price.php?filled=1&T=0.25&strike=50&barrier_down=&barrier_up=&rd=1.000&rf=1.0&spot=50&sigma_0=0.25&sigma_a=0.3&kappa=3&xi=0.5&rho=-0.5&n_s=50&n_v=15&n_t=20

In [22]:
BS(50,52.5,T,0.2523194648043028), IV(1.5178251869083184,50,52.5,T)

(1.517825186908322, 0.25231946480430256)

In [52]:
%timeit py_lets_be_quickly_rational.implied_volatility_from_a_transformed_rational_guess_with_limited_iterations(1.5178251869083184, 50, 52.5, T, 1, 1)

10.8 µs ± 10.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [55]:
800000 * 3 * 5 * 10*10**(-6)

120.0

In [56]:
%timeit np.exp(3.456)

311 ns ± 1.11 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [23]:
modelParam = np.array([
    [1.0, 0.25**2, 3.0, 0.3**2, 0.4, -0.6],
    [1.0, 0.35**2, 3.0, 0.25**2, 0.4, -0.5],
    [1.0, 0.3**2, 2.0, 0.3**2, 0.3, -0.7],
])

In [37]:
T=2
n_reset=8
n_resetPerYear=n_reset//T
n_simStepsPerReset=365//(n_resetPerYear)
n_simStepsPerYear=n_simStepsPerReset*n_resetPerYear
nt=n_simStepsPerReset*n_reset
T_reset=[n_simStepsPerReset*q for q in range(1,n_reset)]

dt=1.0/n_simStepsPerYear
sqrtDt=dt**0.5
na=C.shape[0]
ns=100000
volCorrel=0.002

In [166]:
#simulate correl
U0 = np.linalg.cholesky(C).T
U0 = np.repeat(U0[np.newaxis, :, :], ns, axis=0)
U = np.repeat(U0[np.newaxis, :, :, :], n_reset+1, axis=0)
for i in range(1, 1+n_reset):
    A=U[i-1,:,:,:]
    A=A + np.random.multivariate_normal(np.zeros(na),volCorrel**2*np.eye(na),size=(ns,na))
    U[i,:,:,:]=A/np.linalg.norm(A,axis=1).reshape((ns,1,na))
L = np.transpose(U, axes=[0,1,3,2])
correls = np.matmul(L, U)

correls[1,:2]

array([[[1.        , 0.70390394, 0.30301858],
        [0.70390394, 1.        , 0.50245425],
        [0.30301858, 0.50245425, 1.        ]],

       [[1.        , 0.70237615, 0.30295421],
        [0.70237615, 1.        , 0.50140985],
        [0.30295421, 0.50140985, 1.        ]]])

In [171]:
correlsFlat = np.zeros((n_reset, ns, na*(na-1)//2))
for i in range(n_reset):
    q0, q1 = np.triu_indices(na,1)
    qqq=(np.array([i//na for i in range(na*ns)]), np.tile(q0, ns), np.tile(q1, ns))
    correlsFlat[i] = correls[i,:][qqq].reshape((-1,na))

correlsFlat[1,:2]

array([[0.70390394, 0.30301858, 0.50245425],
       [0.70237615, 0.30295421, 0.50140985]])

In [39]:
# show path/evolution of one correlation matrix by three paths over time

In [40]:
W=np.random.normal(size=(n_reset,ns,na,n_simStepsPerReset)) # you could reduce here because S can do a big step
print(U.shape, W.shape)
W1=np.matmul(L[1:,:,:,:],W)
print(W1.shape)

(9, 100000, 3, 3) (8, 100000, 3, 91)
(8, 100000, 3, 91)


In [41]:
#W1=np.random.normal(size=(n_reset,ns,na,n_simStepsPerReset)) # you could reduce here because S can do a big step
np.mean(W1[1,:,1,1]), np.std(W1[1,:,1,1])

(-0.002931169621649541, 0.9972011464860249)

In [42]:
rho=np.ones_like(W1)
rhoBar=np.ones_like(W1)
for i in range(na):
    rho[:,:,i,:]=modelParam[i,-1]
    rhoBar[:,:,i,:]=(1-modelParam[i,-1]**2)**0.5
W2 = rho*W1 + rhoBar * np.random.normal(size=(n_reset,ns,na,n_simStepsPerReset))

In [43]:
X=np.zeros((n_reset+1,ns,na))
V=np.zeros((n_reset+1,ns,na))

Xr=np.tile(np.log(modelParam[:,0]),(ns,1))
Vr=np.tile(modelParam[:,1],(ns,1))
X[0,:,:] = np.copy(Xr)
V[0,:,:] = np.copy(Vr)

for i in range(1, 1+n_reset):
    for j in range(n_simStepsPerReset):
        sqrtV = np.sqrt(Vr)
        Xr = Xr + sqrtV * sqrtDt * W1[i-1,:,:,j] - 0.5 * Vr * dt
        Vr = np.abs(Vr + modelParam[:,2]*(modelParam[:,3]-Vr)*dt + modelParam[:,4]*sqrtV*sqrtDt*W2[i-1,:,:,j])         
    X[i,:,:] = Xr
    V[i,:,:] = Vr
S=np.exp(X)

In [44]:
kk=4
np.mean(S[kk,:,:]/S[kk-1,:,:],axis=0)-1

array([0.00031886, 0.00053533, 0.00015029])

In [45]:
2*np.std(S[kk,:,:]/S[kk-1,:,:],axis=0)/ns**0.5

array([0.00092252, 0.00080414, 0.00093224])

In [46]:
np.mean(np.maximum(S[1,:,0]-1,0)) # es macht keinen sinn spaeter zeiten zu pruefen, weil die vol bzw var random ist

0.05201152461564503

In [47]:
allWeights = []
n = 11
for i in range(n):
    for j in range(n):
        for k in range(n):
            if i+j+k<=10:
                allWeights.append((i,j,k,10-(i+j+k)))
len(allWeights)
startWeights=np.ones(na)/na

#### no grid for integration ranges based on V0

In [101]:
t1=time.perf_counter()
vol_ref = np.sqrt(modelParam[:,3])
ms = np.array([-0.5, 0, 0.5])
nk=len(ms)
periodDt = T/n_reset
sqrtPeriodDt=periodDt**0.5

Ks_base = np.exp(np.matmul(vol_ref.reshape((na,1)),ms.reshape((1,-1)))*sqrtPeriodDt)
ks = np.array(range(N))
a=np.zeros(na)
b=np.zeros(na)
vv_po = np.zeros((na,N))
for k in range(na):
    kappa, theta, alpha, rho = modelParam[k,2:]
    c1, c2 = cumulants(kappa, theta, theta, periodDt)
    LL=12
    a[k]=c1 - LL*c2**0.5
    b[k]=c1 + LL*c2**0.5
    vv_po[k] = vv_put_orig(a[k],b[k],ks)
xxs = np.pi * np.matmul(1/(b-a).reshape(na,1), ks.reshape(1,N))
heston_a = []
for k in range(na):
    kappa, theta, alpha, rho = modelParam[k,2:]
    heston_a.append(phi_heston2_pre(xxs[k], kappa,theta,alpha,rho,periodDt))


Ks = np.zeros((n_reset,ns,na,nk))
prices = np.zeros((n_reset,ns,na,nk))
vols = np.zeros((n_reset,ns,na,nk))
for i in range(n_reset):
    for j in range(ns):
        Ks[i,j,:,:]=S[i,j,:].reshape((na,1))*Ks_base
        for k in range(na):
            Sk = S[i,j,k]
            Vk = V[i,j,k]            
            xs=np.log(1.0/Ks_base[k,:])
            h = phi_heston2_post(heston_a[k],Vk) * vv_po[k] #vv_put_orig(a[k],b[k],ks)
            q=-1 #put
            numIt=1
            for l in range(len(xs)):
                prices[i,j,k,l] = Ks[i,j,k,l] * np.sum(h * np.exp(1j * (xs[l] - a[k]) * xxs[k])).real
                #print(Sk, Vk, Ks[i,j,k,l], prices[i,j,k,l])
                vols[i,j,k,l] = py_lets_be_quickly_rational.implied_volatility_from_a_transformed_rational_guess_with_limited_iterations(
                    prices[i,j,k,l], Sk, Ks[i,j,k,l], periodDt, q, numIt)
t2=time.perf_counter()
print("time:", t2-t1)

time: 138.23945204200572


In [ ]:
for kk in range(8):
    print(vols[kk,0,0,:], Ks[kk,0,0,:], V[kk,0,0]**0.5)

#### grid for integration ranges

In [109]:
t1=time.perf_counter()
ms = np.array([-2, -1, -0.5, 0, 0.5, 1, 2])
nk=len(ms)
periodDt = T/n_reset
sqrtPeriodDt=periodDt**0.5

volStepSize = 0.025
vol0s = np.arange(volStepSize,1,step=volStepSize)
nv0=len(vol0s)
Ks_base = np.exp(np.matmul(vol0s.reshape((-1,1)), ms.reshape((1,-1))*sqrtPeriodDt))
ks = np.array(range(N))
a=np.zeros((nv0, na))
b=np.zeros((nv0, na))
vv_po = np.zeros((nv0,na,N))
xxs=[]
heston_a = []
for iv0 in range(nv0):
    for k in range(na):
        kappa, theta, alpha, rho = modelParam[k,2:]
        c1, c2 = cumulants(kappa, theta, vol0s[iv0]**2, periodDt)
        LL=12
        a[iv0, k]=c1 - LL*c2**0.5
        b[iv0, k]=c1 + LL*c2**0.5
        vv_po[iv0, k] = vv_put_orig(a[iv0,k],b[iv0,k],ks)
    xxs.append(np.pi * np.matmul(1/(b[iv0]-a[iv0]).reshape(na,1), ks.reshape(1,N)))
    this_a=[]
    for k in range(na):
        kappa, theta, alpha, rho = modelParam[k,2:]
        this_a.append(phi_heston2_pre(xxs[-1][k], kappa,theta,alpha,rho,periodDt))
    heston_a.append(this_a)

#t1=time.perf_counter()
Ks = np.zeros((n_reset,ns,na,nk))
prices = np.zeros((n_reset,ns,na,nk))
vols = np.zeros((n_reset,ns,na,nk))
for i in range(n_reset):
    for j in range(ns):
        for k in range(na):
            Sk = S[i,j,k]
            Vk = V[i,j,k]
            iv0 = min(nv0-1, int((Vk**0.5)//volStepSize))
            Ks[i,j,k,:]=S[i,j,k]*Ks_base[iv0]
            xs=np.log(1.0/Ks_base[iv0,:])
            h = phi_heston2_post(heston_a[iv0][k],Vk) * vv_po[iv0,k] #vv_put_orig(a[k],b[k],ks)
            q=-1 #put
            numIt=1
            for l in range(len(xs)):
                prices[i,j,k,l] = Ks[i,j,k,l] * np.sum(h * np.exp(1j * (xs[l] - a[iv0,k]) * xxs[iv0][k])).real
                if i==n_reset-1 and j<100:
                    print(k, Sk, Vk, Ks[i,j,k,l], prices[i,j,k,l])
                vols[i,j,k,l] = py_lets_be_quickly_rational.implied_volatility_from_a_transformed_rational_guess_with_limited_iterations(
                    prices[i,j,k,l], Sk, Ks[i,j,k,l], periodDt, q, numIt)
t2=time.perf_counter()
print("time:", t2-t1)

0 0.8286333825407016 0.09713539392337177 0.5987102850266119 0.0015988452531316007
0 0.8286333825407016 0.09713539392337177 0.7043517080574939 0.01066600783826413
0 0.8286333825407016 0.09713539392337177 0.7639694616579918 0.02428067914237017
0 0.8286333825407016 0.09713539392337177 0.8286333825407016 0.050071568122818697
0 0.8286333825407016 0.09713539392337177 0.8987705885136958 0.09271134280823973
0 0.8286333825407016 0.09713539392337177 0.9748443495004588 0.15376450521936966
0 0.8286333825407016 0.09713539392337177 1.1468539957190225 0.31848991051673403
1 1.1123025386768024 0.06579446680581907 0.8448740009712614 0.0017302298679401167
1 1.1123025386768024 0.06579446680581907 0.9694098700458755 0.011542928721831972
1 1.1123025386768024 0.06579446680581907 1.038401203519322 0.026520827853836925
1 1.1123025386768024 0.06579446680581907 1.1123025386768024 0.05531655660746353
1 1.1123025386768024 0.06579446680581907 1.191463312401523 0.10333298379647918
1 1.1123025386768024 0.065794466805

0 1.1816859291796697 0.14808463954017306 1.1816859291796697 0.08397240763083993
0 1.1816859291796697 0.14808463954017306 1.3059649232285704 0.16118115509821943
0 1.1816859291796697 0.14808463954017306 1.4433144531791118 0.27344769633678656
0 1.1816859291796697 0.14808463954017306 1.7628682540054044 0.5815318279521484
1 1.5046662481579294 0.03460761226959283 1.231916530485363 0.0032826369969549043
1 1.5046662481579294 0.03460761226959283 1.3614783229890752 0.015763369984225763
1 1.5046662481579294 0.03460761226959283 1.4312828093009158 0.031945350139598726
1 1.5046662481579294 0.03460761226959283 1.5046662481579294 0.06052656266684553
1 1.5046662481579294 0.03460761226959283 1.5818121363809852 0.1058034998637992
1 1.5046662481579294 0.03460761226959283 1.6629133788741393 0.1689316420178193
1 1.5046662481579294 0.03460761226959283 1.8378035056106095 0.3338501611619837
2 1.4088093036626257 0.02134112832389372 1.2125734049242534 0.005309070420008084
2 1.4088093036626257 0.02134112832389372

2 1.5725156163692402 0.03817079520638244 1.9206749110833294 0.3489538125419228
0 1.750996602227145 0.04768828187166036 1.3981991858709368 0.005137545048953161
0 1.750996602227145 0.04768828187166036 1.5646859185462016 0.02328925190865689
0 1.750996602227145 0.04768828187166036 1.6552219569976283 0.04568642287988069
0 1.750996602227145 0.04768828187166036 1.750996602227145 0.08390085858227353
0 1.750996602227145 0.04768828187166036 1.8523129711089252 0.1430074655304514
0 1.750996602227145 0.04768828187166036 1.9594917195009416 0.2249316476479144
0 1.750996602227145 0.04768828187166036 2.192812820943824 0.4429034027439194
1 0.7887592088442653 0.029677376216523887 0.662129455560684 0.002239528624360032
1 0.7887592088442653 0.029677376216523887 0.7226760723315316 0.009044506176699005
1 0.7887592088442653 0.029677376216523887 0.7549949715480891 0.017040015315596787
1 0.7887592088442653 0.029677376216523887 0.7887592088442653 0.03038105514025622
1 0.7887592088442653 0.029677376216523887 0.82

0 0.7685308341173033 0.08800401820639032 0.5693416450698171 0.0017531852583294886
0 0.7685308341173033 0.08800401820639032 0.6614806190533659 0.010381721656435585
0 0.7685308341173033 0.08800401820639032 0.7129994753949777 0.022550919199974682
0 0.7685308341173033 0.08800401820639032 0.7685308341173033 0.044817982137377005
0 0.7685308341173033 0.08800401820639032 0.8283872055611871 0.08082131115243212
0 0.7685308341173033 0.08800401820639032 0.8929054396700128 0.1318453263364311
0 0.7685308341173033 0.08800401820639032 1.037408115326974 0.2692128870300525
1 1.3821740876857598 0.05340251788644521 1.0764382618306743 0.002309866120697976
1 1.3821740876857598 0.05340251788644521 1.219764351215372 0.014085939805772718
1 1.3821740876857598 0.05340251788644521 1.2984323930543014 0.03133233210429336
1 1.3821740876857598 0.05340251788644521 1.3821740876857598 0.06383326557529022
1 1.3821740876857598 0.05340251788644521 1.4713166576013388 0.11742519382986602
1 1.3821740876857598 0.05340251788644

2 0.708793315396045 0.07879024053036357 0.708793315396045 0.03971914663074051
2 0.708793315396045 0.07879024053036357 0.7639970809183687 0.07309062014174604
2 0.708793315396045 0.07879024053036357 0.823500344844033 0.12061901581462657
2 0.708793315396045 0.07879024053036357 0.9567708995383472 0.24814942649025437
0 1.453755068309706 0.06179312728949281 1.1321855855936227 0.0040156553921927565
0 1.453755068309706 0.06179312728949281 1.2829343448999726 0.019772112267465655
0 1.453755068309706 0.06179312728949281 1.3656765013014347 0.040018804990410925
0 1.453755068309706 0.06179312728949281 1.453755068309706 0.07528795943281379
0 1.453755068309706 0.06179312728949281 1.547514214839436 0.13055738598268835
0 1.453755068309706 0.06179312728949281 1.6473203067932018 0.20767893268895116
0 1.453755068309706 0.06179312728949281 1.8666584573482863 0.41374112587221273
1 1.7387888817513395 0.1308367180388749 1.1950509565086391 0.0023602499078759337
1 1.7387888817513395 0.1308367180388749 1.44150661

time: 351.48917712498223


grid for integration range has the same speed

In [ ]:
speed for 100k paths for 3 strikes
1) without implied vol calc: 41s
2) with implied vol calc: 141s
for 5 strikes
2) 250s
for 7 strikes
2) 350s

In [70]:
0.030066957368893407**0.5

0.17339826230067418

In [71]:
 0.001525795928- 2.2121488313629034  +  2.635213925944508

0.4245908905096045

#### compute final implied vols

In [ ]:
t1=time.perf_counter()
ms = np.array([-2, -1, -0.5, 0, 0.5, 1, 2])
nk=len(ms)

Ks = np.zeros((n_reset,ns,na,nk))
prices = np.zeros((n_reset,ns,na,nk))
vols = np.zeros((n_reset,ns,na,nk))
for i in range(n_reset):

    periodDt = (n_reset-i)*T/n_reset
    sqrtPeriodDt=periodDt**0.5

    volStepSize = 0.025
    vol0s = np.arange(volStepSize,1,step=volStepSize)
    nv0=len(vol0s)
    Ks_base = np.exp(np.matmul(vol0s.reshape((-1,1)), ms.reshape((1,-1))*sqrtPeriodDt))
    ks = np.array(range(N))
    a=np.zeros((nv0, na))
    b=np.zeros((nv0, na))
    vv_po = np.zeros((nv0,na,N))
    xxs=[]
    heston_a = []
    for iv0 in range(nv0):
        for k in range(na):
            kappa, theta, alpha, rho = modelParam[k,2:]
            c1, c2 = cumulants(kappa, theta, vol0s[iv0]**2, periodDt)
            LL=12
            a[iv0, k]=c1 - LL*c2**0.5
            b[iv0, k]=c1 + LL*c2**0.5
            vv_po[iv0, k] = vv_put_orig(a[iv0,k],b[iv0,k],ks)
        xxs.append(np.pi * np.matmul(1/(b[iv0]-a[iv0]).reshape(na,1), ks.reshape(1,N)))
        this_a=[]
        for k in range(na):
            kappa, theta, alpha, rho = modelParam[k,2:]
            this_a.append(phi_heston2_pre(xxs[-1][k], kappa,theta,alpha,rho,periodDt))
        heston_a.append(this_a)
    
    for j in range(ns):
        for k in range(na):
            Sk = S[i,j,k]
            Vk = V[i,j,k]
            iv0 = min(nv0-1, int((Vk**0.5)//volStepSize))
            Ks[i,j,k,:]=S[i,j,k]*Ks_base[iv0]
            xs=np.log(1.0/Ks_base[iv0,:])
            h = phi_heston2_post(heston_a[iv0][k],Vk) * vv_po[iv0,k] #vv_put_orig(a[k],b[k],ks)
            q=-1 #put
            numIt=1
            for l in range(len(xs)):
                prices[i,j,k,l] = Ks[i,j,k,l] * np.sum(h * np.exp(1j * (xs[l] - a[iv0,k]) * xxs[iv0][k])).real
                if i==n_reset-1 and j<100:
                    print(k, Sk, Vk, Ks[i,j,k,l], prices[i,j,k,l])
                vols[i,j,k,l] = py_lets_be_quickly_rational.implied_volatility_from_a_transformed_rational_guess_with_limited_iterations(
                    prices[i,j,k,l], Sk, Ks[i,j,k,l], periodDt, q, numIt)
t2=time.perf_counter()
print("time:", t2-t1)

In [63]:
# value functions
# state = weight, spot, var, correl, t
# nicht moeglich weil weight unobservable
# geodesic still possible
# replicable with regression?
# dmc

In [97]:
def valueGeodesic(T, w, spot, vol, Kvol, Acorrel, K): # b=\sum w_i S_i
    
    Sbar = np.dot(w, spot)
    kappa = np.log(K / Sbar)
    omega = w*spot/Sbar
    Omega = np.diag(omega)

    m_mid=len(ms)//2
    sigma_atm = vol[:,m_mid]

    A = np.matmul(T**0.5*np.diag(sigma_atm), Acorrel)
    C = np.matmul(A, A.T)

    omegaComega = np.matmul(omega.T, np.matmul(C, omega))
    zstar=kappa*np.matmul(A.T, omega) / omegaComega + kappa**2/2 *\
        np.matmul((np.eye(na) + 2 * np.matmul(A.T, np.matmul(Omega, A)) / omegaComega -
                   3*np.matmul(omega.T,np.matmul(C, np.matmul(Omega, np.matmul(C, omega)))) / omegaComega**2 * np.eye(na) ),
                 np.matmul(A.T, omega) / omegaComega)
    Kstar=spot*np.exp(np.matmul(A, zstar))

    sigmaStar=np.zeros(na)
    for i in range(na):
        sigmaStar[i] = interpolate.interp1d(Kvol[i,:], vol[i,:])(Kstar[i])

    print(Kstar, sigmaStar)

    M=Sbar
    V2 = 0
    for i in range(na):
        for j in range(na):
            V2+=w[i]*w[j]*spot[i]*spot[j]*np.exp(C[i,j])
    m=2*np.log(M)-0.5*np.log(V2)
    v2=np.log(V2)-2*np.log(M)
    d1=(m-np.log(K)+v2)/v2**0.5
    d2=d1-0.5*v2**0.5
    priceBasket = M * norm.cdf(d1) - K * norm.cdf(d2)
    return priceBasket


nri=0
nsi=1

T = (n_reset-nri)*periodDt
w=np.ones(na)/na
spot=S[nri,nsi]
vol=vols[nri,nsi]
Kvol = Ks[nri,nsi]
A = L[nri, nsi] #correl
K=1

valueGeodesic(T, w, spot, vol, Kvol, A, K)


[1. 1. 1.] [0.26074985 0.31872221 0.29623873]


0.06748250771984399

In [98]:
w, spot

(array([0.33333333, 0.33333333, 0.33333333]), array([1., 1., 1.]))

In [99]:
def policyGeodesic(nri, nsi, w, K):    
    T = (n_reset-nri)*periodDt
    spot=S[nri,nsi]
    vol=vols[nri,nsi]
    Kvol = Ks[nri,nsi]
    A = L[nri, nsi]

    B = np.dot(w, spot)
    bestWeight = w
    bestValue = valueGeodesic(T, w, spot, vol, Kvol, A, K)
    percW = w * spot / B
    for i in range(na):
        for j in range(na):
            if i!=j:
                move = min(0.05, 1.0-percW[i], percW[j])
                if move>1e-8:
                    percW_adj = deepcopy(percW)
                    percW_adj[i]+=move
                    percW_adj[j]-=move
                    w_adj = P * percW_adj / spot
                    v = valueGeodesic(T, w_adj, spot, vol, Kvol, A, K)
                    print(i,j,v,w_adj, percW_adj)
                    if v>bestValue:
                        bestValue=v
                        bestWeight=w_adj
    return bestWeight, bestValue

In [110]:
policyGeodesic(nri, nsi, w, K)

[1. 1. 1.] [0.26074985 0.31872221 0.29623873]


NameError: name 'deepcopy' is not defined

In [ ]:
nri=n_reset-1
for w in weightGrid
state=(w,s,v**0.5,correl) #K fixed, t not needed because we make a seperate regression at each time
samples = num weight x num remaining states
oder for each weight a single regression
wiederholen mit nn


In [187]:
#states_without_weights
nri=n_reset-1
state = np.concatenate([S[nri], np.sqrt(V[nri]), correlsFlat[nri]], axis=1)
state.shape

(100000, 9)

In [188]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
X = PolynomialFeatures(degree=2).fit_transform(states)
X.shape

ModuleNotFoundError: No module named 'sklearn'

In [176]:
w = allWeights[0]
payoff = np.maximum(w*)

286

In [178]:
300*10000

3000000

In [ ]:
fuer value: eine regression fuer each action
fuer gd: normalerweise indicator(beta X > 0); hier exponential form and nn

In [182]:
aa=np.zeros((int(300*ns),50))

In [184]:
aaa=np.matmul(aa.T,aa)

In [185]:
np.linalg.inv(aaa)

LinAlgError: Singular matrix

In [186]:
aaa.shape

(50, 50)

In [200]:
sig=0.2
T=1
n_t=365
dt=T/n_t
n_s=10000

W=np.random.normal(size=(n_s, n_t))
S=np.exp(np.cumsum(sig*dt**0.5*W - 0.5*sig**2*dt, axis=1))
A=np.cumsum(S,axis=1)/np.tile(np.arange(1,n_t+1), (n_s,1))

In [206]:
t_ex = [100, 200, 300]
y=np.maximum(1-A[:,-1], 0) #put
for i in t_ex:
    X = PolynomialFeatures(2).fit_transform(np.vstack([S[:,i], A[:,i]]).T)
    cv = LinearRegression().fit(X, y).predict(X)
    ee = np.maximum(1-A[:,i], 0)
    y = np.where(cv > ee, y, ee)
np.mean(y)

NameError: name 'PolynomialFeatures' is not defined

In [205]:
A_grid = np.linspace(0.1, 2, 0.05)
y=np.maximum(1-A[:,-1], 0) #put
for i in t_ex:
    X = PolynomialFeatures(2).fit_transform(np.vstack([S[:,i], A[:,i]]).T)
    cv = LinearRegression().fit(X, y).predict(X)
    ee = np.maximum(1-A[:,i], 0)
    y = np.where(cv > ee, y, ee)
np.mean(y)

(2, 10000)